# Mechanisms-of-Action-Prediction-baseline-lightGBM

## debug

In [122]:
debug = True

## import libraries

In [123]:
import pandas as pd
import numpy as np
import copy

import lightgbm as lgb

## load data

In [124]:
if debug == True:
    train_features = pd.read_csv("../input/train_features.csv", index_col=["sig_id"])
    train_targets_scored = pd.read_csv("../input/train_targets_scored.csv", index_col=["sig_id"])
    train_targets_nonscored = pd.read_csv("../input/train_targets_nonscored.csv", index_col=["sig_id"])
    test_features = pd.read_csv("../input/test_features.csv", index_col=["sig_id"])
    submission = pd.read_csv("../output/sample_submission.csv")

In [125]:
if debug == False:
    train_features = pd.read_csv('../input/lish-moa/train_features.csv', index_col=["sig_id"])
    train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv', index_col=["sig_id"])
    train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv', index_col=["sig_id"])
    test_features = pd.read_csv('../input/lish-moa/test_features.csv', index_col=["sig_id"])
    submission = pd.read_csv('../input/lish-moa/sample_submission.csv')

## preprocess

In [126]:
import category_encoders as ce

object_cols = ["cp_type", "cp_dose"]
ce_oe = ce.OrdinalEncoder(cols=object_cols, handle_unknown='impute')
train_features = ce_oe.fit_transform(train_features)
test_features = ce_oe.fit_transform(test_features)

## train models and prediction

In [127]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

FOLD_NUM = 5
mskf = MultilabelStratifiedKFold(n_splits=FOLD_NUM, random_state=42)
scores = []
feature_importance_df = pd.DataFrame()
target_cols =  train_targets_scored.columns
pred_cv = np.zeros((len(test_features), len(target_cols)))
num_round = 10000

/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [128]:
train_X = train_features
train_y = train_targets_scored
test_X = test_features

In [129]:
def multiclass_log_loss(y_true, y_pred, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    https://www.kaggle.com/wiki/MultiClassLogLoss

    idea from this post:
    http://www.kaggle.com/c/emc-data-science/forums/t/2149/is-anyone-noticing-difference-betwen-validation-and-leaderboard-error/12209#post12209

    Parameters
    ----------
    y_true : array, shape = [n_samples]
    y_pred : array, shape = [n_samples, n_classes]

    Returns
    -------
    loss : float
    """
    predictions = np.clip(y_pred, eps, 1 - eps)

    # normalize row sums to 1
    predictions /= predictions.sum(axis=1)[:, np.newaxis]

    actual = np.zeros(y_pred.shape)
    rows = actual.shape[0]
    actual[np.arange(rows), y_true.astype(int)] = 1
    vsota = np.sum(actual * np.log(predictions))
    return -1.0 / rows * vsota

In [133]:
params = {
'boosting_type': 'gbdt',
'objective': 'multiclass',
'num_class': 206,
'lambda_l1': 0.001, 
'lambda_l2': 0.001,
'num_leaves': 50, 
'max_depth': 6, # 2**6 = 64 - Let's set 50 to prevent overfitting
'feature_fraction': 0.4,
'subsample': 0.4, 
'min_child_samples': 10,
'learning_rate': 0.01,
'num_iterations': 100, #700
#'early_stopping_rounds': 100,
'random_state': 42}

In [134]:
from lightgbm import LGBMClassifier
from sklearn.multiclass import OneVsRestClassifier

#from sklearn.metrics import log_loss
# If you want to avoid the OneVsRestClassifier magic switch
# from sklearn.multioutput import MultiOutputClassifier

clf_multilabel = OneVsRestClassifier(LGBMClassifier(**params))

In [136]:
for i, (tdx, vdx) in enumerate(mskf.split(train_X, train_y)):
        print(f'Fold : {i}')
        X_train, X_valid, y_train, y_valid = train_X.iloc[tdx], train_X.iloc[vdx], train_y.values[tdx], train_y.values[vdx]
        X_train_lgb = copy.deepcopy(X_train)
        X_valid_lgb = copy.deepcopy(X_valid)
        for obj_col in object_cols:
            X_train_lgb[obj_col] = X_train_lgb[obj_col].astype("category")
            X_valid_lgb[obj_col] = X_valid_lgb[obj_col].astype("category")
        #lgb_train = lgb.Dataset(X_train_lgb, y_train)
        #lgb_valid = lgb.Dataset(X_valid_lgb, y_valid)
        #model = lgb.train(params, lgb_train, num_boost_round=num_round,valid_names=["train", "valid"], valid_sets=[lgb_train, lgb_valid],early_stopping_rounds=50, verbose_eval=10000)
        model = clf_multilabel.fit(X_train_lgb, y_train)
        va_pred = model.predict_proba(X_valid_lgb)#, num_iteration=model.best_iteration)
        score_ = multiclass_log_loss(y_valid, va_pred)
        scores.append(score_)
        print(socre_)
        if debug == False:
            lgb_submission = model.predict_proba(test_X)#, num_iteration=gbc.best_iteration)
            pred_cv += lgb_submission/FOLD_NUM
        
print(np.mean(scores))

Fold : 0


/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/sklearn/multiclass.py:75: UserWarning: Label not 82 is present in all training examples.
  str(classes[c]))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [140]:
print(va_pred.shape,y_valid.shape)

(206,) (4763, 206)


In [ ]:
if debug == False:
    target_cols =  train_targets_scored.columns
    submission[target_cols] = pred_cv
    submission.to_csv("submission.csv", index=False)
    submission.head()